# Big Data Modeling and Management Assigment


## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_ 

#### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions

#### Connection details to the neo4j database
```
Host: rhea.isegi.unl.pt:7474  
Username: neo4j  
Password: F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ 
Connect URL : bolt://rhea.isegi.unl.pt:7687
```


#### Questions


0. __Example Question__ _How many beers does the database contain?_
1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers?
    1. Which `Country` has the most reviews for its beers? 
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV?
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
1. Using Graph Algorithms answer the questions:
    1. Which Countries are most similiar when it comes to the most produced Beer styles
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why?


 Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
##### Groups  

Groups should have 4 to 5 people  
You should register your group on moodle. An email will be going out to everyone with the credentials for the database to use when storing the results.


##### Submission      

Submission of the query results to be done to the group's redis database (explained on the first class, credentials sent via email).  
The following format is expected:
```
    >>> redis.set("0", "358873")
```

This result should be the anwser of the group to question 0

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of May 2nd**

##### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

**Note:**
Remember the Neo4j is a shared database and when creating in-memory graphs please use your group's prefix.  
Ex. Instead of `my-graph` as the name of your graph please use `group0-my-graph`.

In [4]:
import py2neo
from pprint import pprint

username="neo4j"
password="F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ"
host="rhea.isegi.unl.pt"
port="7474"

group_24_graph = py2neo.Graph(f"http://{username}:{password}@{host}:{port}")
group_24_graph.run("MATCH () RETURN count(*)").data()

[{'count(*)': 9647598}]

## Question 4

In [6]:
# Question 4

# Establish the relationship between country, breweries and beers nodes so the brewery name and the total number of beers can be
# returned, only retrieving the name of brewery with the highest number of beers.
group_24_graph.run("""
        MATCH (country:Country {country_digit: 'PT'})-[:FROM]-(brewery:Breweries)-[:BREWED_AT]-(beer:Beers)
        RETURN brewery.name as brewery, count(*) as num_beers
        ORDER BY num_beers DESC
        LIMIT 1
""").data()

[{'brewery': 'Dois Corvos Cervejeira', 'num_beers': 40}]

## Question 5

In [7]:
# Question 5

# Establish the relationship between breweries, beers and reviews nodes so the brewery name and the total number of reviews can
# be returned, only retrieving the name of the beer with the highest number of reviews. 
group_24_graph.run("""
        MATCH (b:Breweries {name: 'Dois Corvos Cervejeira'})-[:BREWED_AT]-(be:Beers)-[:ABOUT]-(rev:Reviews)
        RETURN be.name as beer, count(*) as num_reviews
        ORDER BY num_reviews DESC
        LIMIT 1
""").data()

[{'beer': 'Finisterra', 'num_reviews': 10}]

### Question 9.A

In [10]:
# Question 9.A
# Create a graph that makes the union between all countries' ids and beer styles' ids, including duplicates. The last query
# makes the relationship between the country and style nodes, making the count of the styles for each country, ordering it in a 
# descending order, frrom the most produced style until the tenth one. it retrieves the id of the countries as the source nodes
# and the id of the top 10 styles as the target nodes, to compare the similarity of the countries in terms of the top 10 most 
# produced beer styles.

group_24_graph.run(f"""                                    
    CALL gds.graph.create.cypher(
        'graph-9.A',
        'MATCH (c:Country) return id(c) as id UNION ALL MATCH (s:Style) return id(s) as id',
        'MATCH (c:Country)-[:FROM]-(brewery:Breweries)-[:BREWED_AT]-(be:Beers)-[:OF_TYPE]-(s:Style) WITH c as country, s as style, count(*) as count_style ORDER BY count_style DESC WITH country, collect(style)[..10] as top_10 UNWIND top_10 as top_10_styles RETURN id(country) as source, id(top_10_styles) as target'
    )
""").data()

[{'nodeQuery': 'MATCH (c:Country) return id(c) as id UNION ALL MATCH (s:Style) return id(s) as id',
  'relationshipQuery': 'MATCH (c:Country)-[:FROM]-(brewery:Breweries)-[:BREWED_AT]-(be:Beers)-[:OF_TYPE]-(s:Style) WITH c as country, s as style, count(*) as count_style ORDER BY count_style DESC WITH country, collect(style)[..10] as top_10 UNWIND top_10 as top_10_styles RETURN id(country) as source, id(top_10_styles) as target',
  'graphName': 'graph-9.A',
  'nodeCount': 313,
  'relationshipCount': 1365,
  'createMillis': 1152}]

In [11]:
# Question 9.A
# It finds the nodes with higher similarity between them, in other words, the countries which have a higher similarity. To do
# that, the similarity score is ordered in descending order to output the most similar two countries. In the end, we found out
# that Iran and Libya are two examples of the most similar countries when it comes to beer styles, scoring the highest value for
# similarity, which is 1.

group_24_graph.run("""  
    CALL gds.nodeSimilarity.stream(
      'graph-9.A'
    ) YIELD
      node1,
      node2,
      similarity
    RETURN 
      gds.util.asNode(node1).country_digit as country_1,
      gds.util.asNode(node2).country_digit as country_2,
      similarity
    ORDER BY similarity DESC
    LIMIT 1
""").data()

[{'country_1': 'IR', 'country_2': 'LY', 'similarity': 1.0}]

## Question 9C

In [8]:
# Question 9.C
# Create a graph that makes the union between all Reviews ids and beer styles ids, including duplicates. The last query makes 
# the relationship between the reviews and username nodes. It retrieves the id of the reviews as the source nodes and the id 
# of the usernames as the target nodes, to compare the degree of influence a user has, so as more relationships as user has to
# the reviews nodes, the higher the score and more influence he/she has.

group_24_graph.run(f"""                                    
    CALL gds.graph.create.cypher(
        'graph-9.C',
        'MATCH (rev:Reviews) RETURN id(rev) as id UNION ALL MATCH (u:Username) RETURN id(u) as id',
        'MATCH (rev:Reviews)-[:MADE]-(u:Username) RETURN id(rev) as source, id(u) as target'
    )
""").data()

[{'nodeQuery': 'MATCH (rev:Reviews) RETURN id(rev) as id UNION ALL MATCH (u:Username) RETURN id(u) as id',
  'relationshipQuery': 'MATCH (rev:Reviews)-[:MADE]-(u:Username) RETURN id(rev) as source, id(u) as target',
  'graphName': 'graph-9.C',
  'nodeCount': 9238064,
  'relationshipCount': 9073128,
  'createMillis': 32750}]

In [9]:
# Question 9.C
# It finds the node with higher score, in other words, the user which has made more reviews, thus the most influential user, so
# with the most relationships to the reviews nodes. To find it, the score is ordered in descending order to output the username
# with the higher score. In the end, it was found out that Sammy is the user with the highest score, being deemed to be the most
# influential user, with a score of 1759.37.

group_24_graph.run("""  
    CALL gds.pageRank.stream(
      'graph-9.C'
    ) YIELD
      nodeId,
      score
    RETURN 
      gds.util.asNode(nodeId).user_name as username,
      ROUND(score,2) as score
    ORDER BY score DESC
    LIMIT 1
""").data()

[{'username': 'Sammy', 'score': 1759.37}]